# Neoclassical transport calculations for axisymmetric plasmas

## Define particles in our simulation

In [ ]:
import astropy.units as u

from plasmapy.particles import IonizationStateCollection

all_species = IonizationStateCollection(
    {
        "H": [0, 1],
        #      "D": [0, 1],   raises ParticleError, why?
        "C": [0, 0, 0, 0, 0, 0, 1],
    },
    n0=1e20 * u.m ** -3,
    abundances={"H": 1, "C": 0.08},
    T_e=10 * u.eV,
)

## `SymbolicEquilibrium`

In [ ]:
from plasmapy.plasma.symbolicequilibrium import SymbolicEquilibrium

params = {"aspect_ratio": 0.32, "A": -0.155, "elongation": 1.7, "triangularity": 0.33}
eq = SymbolicEquilibrium(**params, B0=5.2, config="single-null")
eq.plot();

## `FluxSurface`

In [ ]:
ax = eq.plot()
flux_surface = eq.get_flux_surface(-0.01)
flux_surface.plot(ax, n=True);

## Multiple flux surfaces - radial grid

In [ ]:
ax = eq.plot()
for fs in eq.get_multiple_flux_surfaces(np.linspace(-0.037228, 0, 50)):
    fs.plot(ax)
ax.legend().remove()

## `FlowCalculator`

In [ ]:
from plasmapy.transport.flows import FlowCalculator

density_gradient = {
    "H 1+": 1e18 * u.m ** -3 / u.m,
    "C 1+": 1e18 * u.m ** -3 / u.m,
}
temperature_gradient = {
    "H 1+": -10 * u.K / u.m,
    "C 1+": -10 * u.K / u.m,
}

fc = FlowCalculator(all_species, flux_surface, density_gradient, temperature_gradient)

### Bootstrap current

In [ ]:
fc.bootstrap_current.si  # TODO should be A/m^2? missing a magnetic field in the denominator somewhere?

In [ ]:
(fc.bootstrap_current / u.T).to(u.A / u.m ** 2)

### Fluxes

The units are horribly off for each of them atm.

In [ ]:
flux_unit = u.m ** -2 / u.s

In [ ]:
fc._fluxes_BP

In [ ]:
fc._fluxes_BP["H 1+"].particle_flux.to(flux_unit)

In [ ]:
fc._fluxes_BP["H 1+"].heat_flux.to(u.J * flux_unit)

In [ ]:
fc._fluxes_CL

In [ ]:
(fc._fluxes_CL["H 1+"].particle_flux / (u.kg / u.s / u.m ** 4)).to(flux_unit)

In [ ]:
(fc._fluxes_CL["H 1+"].heat_flux / (u.kg / u.s / u.m ** 4)).to(u.J * flux_unit)

In [ ]:
fc._fluxes_PS

In [ ]:
(fc._fluxes_PS["H 1+"].particle_flux / (u.kg / u.s / u.m ** 3)).to(flux_unit)

In [ ]:
(fc._fluxes_PS["H 1+"].heat_flux / (u.kg / u.s / u.m ** 3)).to(u.J * flux_unit)